In [1]:
import pandas as pd
import numpy as np
import librosa
from sklearn.ensemble import IsolationForest

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
train = pd.read_csv('sound/train.csv')
test = pd.read_csv('sound/test.csv')
sample_submission = pd.read_csv('sound/sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sound/train.csv'

In [92]:
train_Sample_list = []
for i in range(len(train)):
    y, sr = librosa.load('sound/train/'+train['SAMPLE_ID'][i]+'.wav')
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec))
    mfcc_list = []
    for e in mfcc:
        mfcc_list.append(np.mean(e))
    train_Sample_list.append(mfcc_list)

In [94]:
train['Sample'] = train_Sample_list

In [96]:
Sample_list = []
for i in range(len(test)):
    y, sr = librosa.load('sound/test/'+test['SAMPLE_ID'][i]+'.wav')
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec))
    mfcc_list = []
    for e in mfcc:
        mfcc_list.append(np.mean(e))
    Sample_list.append(mfcc_list)

In [98]:
test['Sample'] = Sample_list

In [100]:
test = test[['FAN_TYPE', 'Sample']]
test0 = test[test['FAN_TYPE'] == 0]
test2 = test[test['FAN_TYPE'] == 2]

In [167]:
lf0 = IsolationForest(n_estimators=1000, max_samples= 512, random_state=230426, n_jobs=-1, contamination='auto')
lf2 = IsolationForest(n_estimators=1000, max_samples= 512, random_state=230426, n_jobs=-1, contamination='auto')

In [168]:
lf0.fit(train[train['FAN_TYPE'] == 0]['Sample'].tolist())
lf2.fit(train[train['FAN_TYPE'] == 2]['Sample'].tolist())

IsolationForest(max_samples=512, n_estimators=1000, n_jobs=-1,
                random_state=230426)

In [169]:
pred0 = lf0.predict(test0['Sample'].tolist())
pred2 = lf2.predict(test2['Sample'].tolist())

In [170]:
pred0 = np.where(pred0 == 1, 0, pred0)
pred0 = np.where(pred0 == -1, 1, pred0)

pred2 = np.where(pred2 == 1, 0, pred2)
pred2 = np.where(pred2 == -1, 1, pred2)

In [171]:
test0['LABEL'] = pred0
test2['LABEL'] = pred2

In [172]:
result = pd.concat([test0, test2]).sort_index()

In [173]:
sample_submission['LABEL'] = result['LABEL']

In [174]:
sample_submission.to_csv('result2.csv', index=False)